In [6]:
# !pip install --upgrade pip
# !pip install --upgrade https://storage.googleapis.com/tensorflow/linux/cpu/tensorflow-0.11.0-cp27-none-linux_x86_64.whl
!pip install --upgrade protobuf==3.0.0

Requirement already up-to-date: protobuf==3.0.0 in /opt/conda/envs/python2/lib/python2.7/site-packages
Requirement already up-to-date: setuptools in /opt/conda/envs/python2/lib/python2.7/site-packages (from protobuf==3.0.0)
Requirement already up-to-date: six>=1.9 in /opt/conda/envs/python2/lib/python2.7/site-packages (from protobuf==3.0.0)
Requirement already up-to-date: appdirs>=1.4.0 in /opt/conda/envs/python2/lib/python2.7/site-packages (from setuptools->protobuf==3.0.0)
Requirement already up-to-date: packaging>=16.8 in /opt/conda/envs/python2/lib/python2.7/site-packages (from setuptools->protobuf==3.0.0)
Requirement already up-to-date: pyparsing in /opt/conda/envs/python2/lib/python2.7/site-packages (from packaging>=16.8->setuptools->protobuf==3.0.0)


In [3]:
!rm *.csv
!wget https://vincentarelbundock.github.io/Rdatasets/csv/datasets/iris.csv
!wget http://download.tensorflow.org/data/iris_training.csv
!wget http://download.tensorflow.org/data/iris_test.csv

rm: cannot remove ‘*.csv’: No such file or directory
--2017-01-27 06:28:33--  https://vincentarelbundock.github.io/Rdatasets/csv/datasets/iris.csv
Resolving vincentarelbundock.github.io (vincentarelbundock.github.io)... 151.101.52.133
Connecting to vincentarelbundock.github.io (vincentarelbundock.github.io)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4821 (4.7K) [text/csv]
Saving to: ‘iris.csv’

iris.csv            100%[=====================>]   4.71K  --.-KB/s   in 0s     

2017-01-27 06:28:34 (62.2 MB/s) - ‘iris.csv’ saved [4821/4821]

--2017-01-27 06:28:34--  http://download.tensorflow.org/data/iris_training.csv
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.3.176, 2607:f8b0:400a:806::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.3.176|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2194 (2.1K) [text/csv]
Saving to: ‘iris_training.csv’

iris_training.csv   100

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

In [2]:
# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TEST = "iris_test.csv"

# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)

test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[10, 20, 10],
                                            n_classes=3,
                                            model_dir="/tmp/iris_model")
classifier.fit(x=training_set.data,
           y=training_set.target,
           steps=2000)

Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.


Estimator(params={'enable_centered_bias': True, 'activation_fn': <function relu at 0x7f7cb18ba0c8>, 'weight_column_name': None, 'hidden_units': [10, 20, 10], 'feature_columns': [_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None)], 'n_classes': 3, 'optimizer': 'Adagrad', 'dropout': None, 'gradient_clip_norm': None, 'num_ps_replicas': 0})

In [36]:
!rm -rf /tmp/iris_exported*
from tensorflow.contrib.learn.python.learn.utils import export
from tensorflow.contrib.session_bundle import exporter

def my_input_fn():
    # Return features tensors
    return {
        "": tf.placeholder(tf.float32, shape=[None, 4])
    }, None

def my_signature_fn(examples,features,predictions):
    return None,{
        "inputs": exporter.generic_signature({"features": examples}),
        "outputs": exporter.generic_signature({"score": predictions})
    }

classifier.export(
    "/tmp/iris_exported_model",
    input_fn=my_input_fn,
    input_feature_key="",
    use_deprecated_input_fn=False,
    signature_fn=my_signature_fn
)

Instructions for updating:
The signature of the input_fn accepted by export is changing to be consistent with what's used by tf.Learn Estimator's train/evaluate. input_fn and input_feature_key will become required args. use_deprecated_input_fn will default to False and be removed. default_batch_size will also be removed since it will now be a part of the input_fn.


'/tmp/iris_exported_model/00002000'

In [4]:
# XXX: this only works for tf 0.11

import tensorflow as tf
from tensorflow.core.protobuf import meta_graph_pb2
from tensorflow.contrib.session_bundle import manifest_pb2

def read_serving_signatures(meta_file):
    tf.reset_default_graph()
    with tf.Session() as sess:
        new_saver = tf.train.import_meta_graph(meta_file)

        graph = tf.get_default_graph()
        # print(graph.get_all_collection_keys())
        pb = graph.get_collection("serving_signatures")[0]
        signatures = manifest_pb2.Signatures()
        pb.Unpack(signatures)
        
        return signatures
    
def read_meta_graph(meta_file):
    meta_graph = meta_graph_pb2.MetaGraphDef()
    with open(meta_file, "rb") as f:
        meta_graph.ParseFromString(f.read())
        
    return meta_graph

In [25]:
# mnist/00000001/export.meta
# sales-forecast-model/export.meta
# /tmp/iris_exported_model/00002000/export.meta

signatures = read_serving_signatures('/tmp/iris_exported_model/00002000/export.meta')
signatures

named_signatures {
  key: "inputs"
  value {
    generic_signature {
      map {
        key: "features"
        value {
          tensor_name: "Placeholder:0"
        }
      }
    }
  }
}
named_signatures {
  key: "outputs"
  value {
    generic_signature {
      map {
        key: "score"
        value {
          tensor_name: "Softmax:0"
        }
      }
    }
  }
}

In [27]:
!(cd /tmp/iris_exported_model/00002000/ && tar -cf iris.tar . && mv iris.tar ~/work)

tar: ./iris.tar: file is the archive; not dumped
